In [7]:
import torch
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import os

In [3]:
im_to_cons = pd.read_csv('process_data/all_ims_guide.csv')

In [4]:
im_to_cons.head()

,im_lat,im_lon,clust_lat,clust_lon,nightlight,consumption,nightlight_bin,images,clust_num,images_renamed
0,-17.125000,35.174999,-17.09515,35.217213,0.0,2.039307,1,-17.125_35.174999.png,0,-17.125_35.174999_0.png
1,-17.133333,35.174999,-17.09515,35.217213,0.0,2.039307,1,-17.133333_35.174999.png,0,-17.133333_35.174999_0.png
2,-17.066666,35.191666,-17.09515,35.217213,0.0,2.039307,1,-17.066666_35.191666.png,0,-17.066666_35.191666_0.png
3,-17.050000,35.199999,-17.09515,35.217213,0.0,2.039307,1,-17.05_35.199999.png,0,-17.05_35.199999_0.png
4,-17.100000,35.199999,-17.09515,35.217213,0.0,2.039307,1,-17.1_35.199999.png,0,-17.1_35.199999_0.png


In [9]:
# turn the data into a PyTorch Tensor
data_transforms = {
    'transform': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

In [11]:
device = torch.device("cpu")

Run this if you want to see how many the model gets right on the train set. Keep in mind that nightlights is not the main objective but just a means to an end.

In [ ]:
# model_ft.eval()   # Set model to evaluate mode

# running_loss = 0.0
# running_corrects = 0
# total = 0

# # Iterate over data.
# for inputs, labels in dataloaders_dict['train']:
#     inputs = inputs.to(device)
#     labels = labels.to(device)

#     # forward
#     # track history if only in train
#     with torch.set_grad_enabled(False):
#         outputs = model_ft(inputs)
#         loss = criterion(outputs, labels)

#         _, preds = torch.max(outputs, 1)

#     # statistics
#     running_loss += loss.item() * inputs.size(0)
#     running_corrects += torch.sum(preds == labels.data)
    
#     total += len(preds)
#     if total == 1000:
#         break
        
# running_corrects.double()/total

In [38]:
from PIL import Image
import matplotlib.pyplot as plt

def filename_to_im_tensor(file):
    im = plt.imread(file)[:,:,:3]
    im = data_transforms['transform'](im)
    return im[None].to(device)

In [13]:
model_ft = torch.load('trained_model.pt')

In [14]:
model_ft.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=3, bias=True)
)

In [26]:
# now we "rip" off the final layers
ripped = model_ft.classifier
del ripped[6]
del ripped[5]
del ripped[4]
ripped

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
)

In [28]:
model_ft

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=2, stride=

In [30]:
unique_ims = im_to_cons.drop_duplicates(subset='images'); unique_ims.shape

(19997, 10)

In [36]:
feats = np.zeros((unique_ims.shape[0],4096))

In [44]:
model_ft.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=2, stride=

In [46]:
# this might take around 45 minutes
model_ft.eval()
for i, im_name in enumerate(unique_ims['images']):
    image = filename_to_im_tensor('process_data/data/ims_malawi_2016/{}'.format(im_name))
    res = np.array(model_ft(image).cpu().detach().numpy().tolist()[0])
    feats[i,:] += res
    if i % 100 == 0:
        print(i, end=', ')

0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000, 6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200, 7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400, 8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600, 9700, 9800, 9900, 10000, 10100, 10200, 10300, 10400, 10500, 10600, 10700, 10800, 10900, 11000, 11100, 11200, 11300, 11400, 11500, 11600, 11700, 11800, 11900, 12000, 12100, 12200, 12300, 12400, 12500, 12600, 12700, 12800, 12900, 13000, 13100, 13200, 13300, 13400, 13500, 13600, 13700, 13800, 13900, 14000, 14100, 14200, 14300, 14400, 14500, 14600, 14700, 14800, 14900, 15000, 15100, 15200, 15300, 15400, 15500, 15600, 15700, 15800,

In [47]:
# save them for safekeeping
np.save('forward_feats_trained_model.npy', feats)

In [49]:
unique_ims.head()

,im_lat,im_lon,clust_lat,clust_lon,nightlight,consumption,nightlight_bin,images,clust_num,images_renamed
0,-17.125000,35.174999,-17.09515,35.217213,0.0,2.039307,1,-17.125_35.174999.png,0,-17.125_35.174999_0.png
1,-17.133333,35.174999,-17.09515,35.217213,0.0,2.039307,1,-17.133333_35.174999.png,0,-17.133333_35.174999_0.png
2,-17.066666,35.191666,-17.09515,35.217213,0.0,2.039307,1,-17.066666_35.191666.png,0,-17.066666_35.191666_0.png
3,-17.050000,35.199999,-17.09515,35.217213,0.0,2.039307,1,-17.05_35.199999.png,0,-17.05_35.199999_0.png
4,-17.100000,35.199999,-17.09515,35.217213,0.0,2.039307,1,-17.1_35.199999.png,0,-17.1_35.199999_0.png


In [50]:
unique_ims = unique_ims[['images']]
unique_ims['feat_index'] = np.arange(len(unique_ims))

In [51]:
unique_ims.head()

,images,feat_index
0,-17.125_35.174999.png,0
1,-17.133333_35.174999.png,1
2,-17.066666_35.191666.png,2
3,-17.05_35.199999.png,3
4,-17.1_35.199999.png,4


In [53]:
im_to_cons = pd.merge(left=im_to_cons, right=unique_ims, on='images')

In [54]:
im_to_cons.head()

,im_lat,im_lon,clust_lat,clust_lon,nightlight,consumption,nightlight_bin,images,clust_num,images_renamed,feat_index
0,-17.125000,35.174999,-17.09515,35.217213,0.0,2.039307,1,-17.125_35.174999.png,0,-17.125_35.174999_0.png,0
1,-17.133333,35.174999,-17.09515,35.217213,0.0,2.039307,1,-17.133333_35.174999.png,0,-17.133333_35.174999_0.png,1
2,-17.066666,35.191666,-17.09515,35.217213,0.0,2.039307,1,-17.066666_35.191666.png,0,-17.066666_35.191666_0.png,2
3,-17.050000,35.199999,-17.09515,35.217213,0.0,2.039307,1,-17.05_35.199999.png,0,-17.05_35.199999_0.png,3
4,-17.100000,35.199999,-17.09515,35.217213,0.0,2.039307,1,-17.1_35.199999.png,0,-17.1_35.199999_0.png,4


In [56]:
group = im_to_cons.groupby(['clust_lat', 'clust_lon'])

In [57]:
num_clusts = len(group); num_clusts

778

In [58]:
x = np.zeros((num_clusts, 4096))
y = []

In [59]:
# this goes through each cluster group and finds all images that are in the cluster
# it aggregates the features for those images across the cluster
for i, g in enumerate(group):
    lat, long = g[0]
    im_sub = im_to_cons[(im_to_cons['clust_lat'] == lat) & (im_to_cons['clust_lon'] == long)].reset_index(drop=True)
    agg_feats = np.zeros((len(im_sub), 4096))
    for j, d in im_sub.iterrows():
        agg_feats[j,:] = feats[d.feat_index]
    agg_feats = agg_feats.mean(axis=0) # averages the features across all images in the cluster
    
    x[i,:] = agg_feats
    y.append(g[1]['consumption'].values[0])

In [60]:
y = np.array(y)
y_log = np.log(y) # try predicting consumption and log consumption

In [61]:
# This is a bunch of code from the Jean et al Github that is modified to work with Python3 and our data

import numpy as np
import pandas as pd
import random
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
from matplotlib.collections import EllipseCollection
import seaborn as sns


def predict_consumption(
    X, y, dimension=None, k=5, k_inner=5, points=10,
        alpha_low=1, alpha_high=5, margin=0.25):
    """
    Plots predicted consumption
    """
    X = reduce_dimension(X, dimension)
    y_hat, r2 = run_cv(X, y, k, k_inner, points, alpha_low, alpha_high)
    return X, y, y_hat, r2


def plot_predictions(country, y, y_hat, r2, margin):
    """
    Plots consumption predictions vs. true values.
    """
    slope, intercept, ymin, ymax, xmin, xmax = compute_plot_params(
        y, y_hat, margin)
    sns.set_style('white')
    plt.figure()
    plt.axis('equal')
    plt.scatter(y, y_hat, edgecolor='k', color='lightblue', s=20, marker='o')
    x_trend = np.array([xmin, xmax]) * 1.5
    y_trend = slope * x_trend + intercept
    plt.plot(x_trend, y_trend, 'b-', linewidth=2,
             color=sns.xkcd_rgb['french blue'])
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    plt.xlabel('Log consumption expenditures', fontsize=14)
    plt.ylabel('Model predictions', fontsize=14)
    plt.title(country.capitalize() + ': $r^2 = {0:.2f}$'.format(r2))
    plt.show()


def compute_plot_params(y, y_hat, margin):
    """
    Computes parameters for plotting consumption predictions vs. true values.
    """
    slope, intercept, _, _, _ = stats.linregress(y, y_hat)
    ymin = min(y_hat) - margin
    ymax = max(y_hat) + margin
    xmin = min(y) - margin
    xmax = max(y) + margin
    return slope, intercept, ymin, ymax, xmin, xmax


def compare_models(
    country_path, survey, percentiles, dimension, k, k_inner, trials,
        poverty_line, multiples):
    """
    Evaluates and plots comparison of transfer learning and nightlight models.
    """
    r2s, r2s_nl = evaluate_percentiles(
        country_path, survey, percentiles, dimension, k, k_inner, trials)
    if survey == 'lsms':
        X, X_nl, y = load_and_reduce_country_by_percentile(
            country_path, survey, 1.0, dimension)
        fractions = compute_fractions(poverty_line, multiples, y)
        plot_percentiles_lsms(percentiles, multiples, r2s, r2s_nl, fractions)
    elif survey == 'dhs':
        plot_percentiles_dhs(percentiles, r2s, r2s_nl)


def load_and_reduce_country_by_percentile(
        country_path, survey, percentile, dimension):
    """
    Loads data for one country up to a certain percentile.
    """
    if survey == 'lsms':
        X, X_nl, y = load_country_lsms(country_path)
    elif survey == 'dhs':
        X, X_nl, y = load_country_dhs(country_path)
    X, X_nl, y = threshold_by_percentile(X, X_nl, y, percentile)
    X = reduce_dimension(X, dimension)
    return X, X_nl, y


def threshold_by_percentile(X, X_nl, y, percentile):
    """
    Threshold data by output percentile.
    """
    threshold = np.percentile(y, q=100*percentile)
    X = X[y <= threshold]
    X_nl = X_nl[y <= threshold]
    y = y[y <= threshold]
    return X, X_nl, y


def evaluate_percentiles(
        country_path, survey, percentiles, dimension, k, k_inner, trials):
    """
    Evaluate transfer learning and nightlight models for each percentile.
    """
    r2s = np.zeros((len(percentiles), trials))
    r2s_nl = np.zeros((len(percentiles), trials))
    for idx, percentile in enumerate(percentiles):
        for trial in xrange(trials):
            X, X_nl, y = load_and_reduce_country_by_percentile(
                country_path, survey, percentile, dimension)
            _, r2 = run_cv(
                X, y, k, k_inner, points=10, alpha_low=0, alpha_high=3,
                randomize=False)
            r2_nl = run_cv_ols(X_nl, y, k)
            r2s[idx, trial] = r2
            r2s_nl[idx, trial] = r2_nl
    r2s = r2s.mean(axis=1)
    r2s_nl = r2s_nl.mean(axis=1)
    return r2s, r2s_nl


def run_cv_ols(X, y, k):
    """
    Runs OLS in cross-validation to compute r-squared.
    """
    r2s = np.zeros((k,))
    kf = cross_validation.KFold(n=y.size, n_folds=k, shuffle=True)
    fold = 0
    for train_idx, test_idx in kf:
        r2s, fold = evaluate_fold_ols(X, y, train_idx, test_idx, r2s, fold)
    return r2s.mean()


def evaluate_fold_ols(X, y, train_idx, test_idx, r2s, fold):
    """
    Evaluates one fold of outer CV using OLS.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train, X_test = scale_features(X_train, X_test)
    y_test_hat = train_and_predict_ols(X_train, y_train, X_test)
    r2 = stats.pearsonr(y_test, y_test_hat)[0] ** 2
    if np.isnan(r2):
        r2 = 0
    r2s[fold] = r2
    return r2s, fold + 1


def train_and_predict_ols(X_train, y_train, X_test):
    """
    Trains OLS model and predicts test set.
    """
    ols = linear_model.LinearRegression()
    ols.fit(X_train, y_train)
    y_hat = ols.predict(X_test)
    return y_hat


def compute_fractions(poverty_line, multiples, y):
    """
    Computes the fraction of clusters below each multiple of the poverty line.
    """
    fractions = np.zeros((len(multiples),))
    for idx, multiple in enumerate(multiples):
        fractions[idx] = (
            np.exp(y) <= poverty_line * multiple).sum() / float(y.size)
    return fractions


def plot_percentiles_lsms(percentiles, multiples, r2s, r2s_nl, fractions):
    """
    Plots transfer learning model vs. nightlights model at each percentile.
    """
    sns.set_style('white')
    plt.figure(figsize=(6, 6))
    lines = []
    percentiles = [100 * x for x in percentiles]
    for idx, multiple in enumerate(multiples):
        lines.append(
            plt.axvline(
                100 * fractions[idx], color='r', linestyle='dashed',
                linewidth=3.0 / (idx + 1),
                label=str(multiple) + 'x poverty line'))
    line_legend = plt.legend(
        handles=lines, title='Poverty line multiples:', loc='upper right',
        bbox_to_anchor=(0.5, 1), fontsize=10)
    plt.gca().add_artist(line_legend)
    curve1, = plt.plot(percentiles, r2s, label='Transfer learning')
    curve2, = plt.plot(percentiles, r2s_nl, label='Nightlights')
    plt.legend(
        handles=[curve1, curve2], loc='upper right',
        bbox_to_anchor=(0.5, 0.65))
    plt.xlabel('Poorest percent of clusters used', fontsize=14)
    plt.ylabel('$r^2$', fontsize=18)
    plt.show()


def plot_percentiles_dhs(percentiles, r2s, r2s_nl):
    """
    Plots transfer learning model vs. nightlights model at each
    """
    sns.set_style('white')
    plt.figure(figsize=(6, 6))
    percentiles = [100 * x for x in percentiles]
    plt.plot(percentiles, r2s)
    plt.plot(percentiles, r2s_nl)
    plt.legend(['Transfer learning', 'Nightlights'], loc='upper center')
    plt.xlabel('Poorest percent of clusters used', fontsize=14)
    plt.ylabel('$r^2$', fontsize=18)
    plt.show()


def run_randomization_test(
    country_names, country_paths, survey, dimension, k, k_inner, points,
        alpha_low, alpha_high, trials):
    """
    Runs randomization test for a set of countries.
    """
    data = load_data(country_paths, survey, dimension)
    true_r2s = compute_true_r2s(
        data, k, k_inner, points, alpha_low, alpha_high)
    shuffled_r2s = compute_shuffled_r2s(
        data, k, k_inner, points, alpha_low, alpha_high, trials)
    plot_shuffled_distributions(country_names, survey, shuffled_r2s, true_r2s)


def compute_true_r2s(data, k, k_inner, points, alpha_low, alpha_high):
    """
    Uses data to compute true model r2s.
    """
    true_r2s = []
    for (X, y) in data:
        _, r2 = run_cv(X, y, k, k_inner, points, alpha_low, alpha_high)
        true_r2s.append(r2)
    return true_r2s


def compute_shuffled_r2s(
        data, k, k_inner, points, alpha_low, alpha_high, trials):
    """
    Uses data to compute shuffled model r2s.
    """
    shuffled_r2s = np.zeros((len(data), trials))
    for data_idx, (X, y) in enumerate(data):
        for trial in range(trials):
            _, shuffled_r2s[data_idx, trial] = run_cv(
                X, y, k, k_inner, points, alpha_low, alpha_high,
                randomize=True)
    return shuffled_r2s


def plot_shuffled_distributions(country_names, survey, shuffled_r2s, true_r2s):
    """
    Plots shuffled r2 distributions vs. true model r2s.
    """
    colors = sns.color_palette('husl', len(country_names))
    sns.set_style('white')
    plt.figure(figsize=(8, 8))
    frame = plt.gca()
    for i in xrange(len(country_names)):
        plt.axvline(
            true_r2s[i], color=colors[i], linestyle='dashed', linewidth=2,
            label=country_names[i].capitalize() + ': $r^2={0:.2f}$'.format(
                true_r2s[i]))
    plt.legend(
        title='True {} Models:'.format(survey.upper()), loc='lower right',
        bbox_to_anchor=(0.6, 0.45), fontsize=12)
    for i in xrange(len(country_names)):
        sns.kdeplot(shuffled_r2s[i, :], shade=True, color=colors[i])
    plt.xlim((0, max(true_r2s) + 0.05))
    plt.xlabel('$r^2$', fontsize=18)
    plt.ylabel('Randomized $r^2$ distribution', fontsize=14)
    frame.yaxis.set_ticklabels([])
    plt.show()


def evaluate_models(
    country_names, country_paths, survey, dimension, k, trials, points,
        alpha_low, alpha_high, cmap='Greens'):
    """
    Evaluates in- and out-of-country performance for models trained on set of
    countries given.
    """
    n = len(country_names)
    performance_matrix = np.zeros((trials, n, n))
    data = load_data(country_paths, survey, dimension=None)
    for trial in xrange(trials):
        for in_idx in xrange(n):
            performance_matrix[trial, :, in_idx] = compute_column(
                country_names[in_idx], data, in_idx, dimension, k, points,
                alpha_low, alpha_high)
    performance_matrix = performance_matrix.mean(axis=0)
    plot_model_performance(performance_matrix, country_names, cmap)
    return np.around(performance_matrix, decimals=2)


def compute_column(
    country_name, data, in_idx, dimension, k, points, alpha_low,
        alpha_high):
    """
    Evaluates in- and out-of-country performance for model trained on one
    country.
    """
    in_data = data[in_idx]
    out_data = list(data)
    out_data.pop(in_idx)
    if country_name == 'pooled':
        r2s_in, r2s_out = evaluate_pooled_r2s(
            out_data, dimension, k, points, alpha_low, alpha_high)
    else:
        r2s_in, r2s_out = evaluate_model_r2s(
            in_data, out_data, dimension, k, points, alpha_low, alpha_high)
    max_idx = np.argmax(r2s_in)
    r2s = list(r2s_out[:, max_idx])
    r2s.insert(in_idx, r2s_in[max_idx])
    r2s.reverse()
    r2s = np.array(r2s)
    return r2s


def evaluate_pooled_r2s(all_data, dimension, k, points, alpha_low, alpha_high):
    """
    Evaluates in- and out-of-country r2s for the pooled model.
    """
    X_all_full, y_all = [list(data) for data in zip(*all_data)]
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s_in = np.zeros((k, points))
    r2s_out = np.zeros((len(all_data), k, points))
    kf = []
    for i in xrange(len(X_all_full)):
        kf.append(list(cross_validation.KFold(
            n=all_data[i][1].size, n_folds=k, shuffle=True)))
    for fold in xrange(k):
        X_train, X_test, y_train, y_test = split_pooled_data(
            kf, fold, X_all_full, y_all)
        X_train, X_test, X_all = reduce_and_scale_features(
            X_train, X_test, X_all_full, dimension)
        r2s_in, r2s_out = evaluate_alphas(
            X_train, X_test, X_all, y_train, y_test, y_all, r2s_in, r2s_out,
            fold, alphas)
    return r2s_in.mean(axis=0), r2s_out.mean(axis=1)


def split_pooled_data(kf, fold, X_all_full, y_all):
    """
    Splits pooled training and test data for each fold.
    """
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    for i in xrange(len(kf)):
        train_ind = kf[i][fold][0]
        test_ind = kf[i][fold][1]
        X_train.append(X_all_full[i][train_ind])
        X_test.append(X_all_full[i][test_ind])
        y_train.append(y_all[i][train_ind])
        y_test.append(y_all[i][test_ind])
    X_train = np.vstack(X_train)
    X_test = np.vstack(X_test)
    y_train = np.hstack(y_train)
    y_test = np.hstack(y_test)
    return X_train, X_test, y_train, y_test


def evaluate_model_r2s(
        in_data, out_data, dimension, k, points, alpha_low, alpha_high):
    """
    Evaluates in- and out-of-country r2s.
    """
    X, y = in_data
    X_out_full, y_out = [list(data) for data in zip(*out_data)]
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s_in = np.zeros((k, points))
    r2s_out = np.zeros((len(out_data), k, points))
    kf = cross_validation.KFold(n=in_data[1].size, n_folds=k, shuffle=True)
    fold = 0
    for train_ind, test_ind in kf:
        X_train, X_test = X[train_ind], X[test_ind]
        y_train, y_test = y[train_ind], y[test_ind]
        X_train, X_test, X_out = reduce_and_scale_features(
            X_train, X_test, X_out_full, dimension)
        r2s_in, r2s_out = evaluate_alphas(
            X_train, X_test, X_out, y_train, y_test, y_out, r2s_in, r2s_out,
            fold, alphas)
        fold += 1
    return r2s_in.mean(axis=0), r2s_out.mean(axis=1)


def evaluate_alphas(
    X_train, X_test, X_out, y_train, y_test, y_out, r2s_in, r2s_out, fold,
        alphas):
    """
    Computes r2 for different regularization constants.
    """
    for idx, alpha in enumerate(alphas):
        ridge = linear_model.Ridge(alpha=alpha)
        ridge.fit(X_train.astype(np.float), y_train.astype(np.float))
        r2s_in[fold, idx] = compute_r2(ridge, X_test, y_test)
        for i in xrange(len(X_out)):
            r2s_out[i, fold, idx] = compute_r2(ridge, X_out[i], y_out[i])
    return r2s_in, r2s_out


def reduce_and_scale_features(X_train, X_test, X_out_full, dimension):
    """
    Reduces dimension and scales in- and out-of-country features.
    """
    pca = PCA(n_components=dimension)
    scaler = StandardScaler(with_mean=True, with_std=False)
    X_train = scaler.fit_transform(pca.fit_transform(X_train))
    X_test = scaler.transform(pca.transform(X_test))
    X_out = []
    for X in X_out_full:
        X_out.append(scaler.transform(pca.transform(X)))
    return X_train, X_test, X_out


def compute_r2(model, X, y):
    """
    Computes model r2.
    """
    y_hat = model.predict(X)
    r2 = stats.pearsonr(y, y_hat)[0] ** 2
    return r2


def plot_circles(data, ax, **kwargs):
    """
    Plots circles for r2 values.
    """
    M = np.array(data)
    xy = np.indices(M.shape)[::-1].reshape(2, -1).T
    w = np.abs(M).ravel()
    h = np.abs(M).ravel()
    a = 0
    circles = EllipseCollection(
        widths=w, heights=h, angles=a, units='x', offsets=xy,
        transOffset=ax.transData, array=M.ravel(), **kwargs)
    ax.add_collection(circles)
    ax.set_xticks(np.arange(M.shape[1]))
    ax.set_xticklabels(data.columns)
    ax.set_yticks(np.arange(M.shape[0]))
    ax.set_yticklabels(data.index, rotation=90)
    return circles


def plot_model_performance(data, country_names, cmap='Greens'):
    """
    Makes plot for in- and out-of-country model performance.
    """
    data = pd.DataFrame(np.flipud(data))
    data.columns = [country.capitalize() for country in country_names]
    data.index = [country.capitalize() for country in country_names]
    sns.set_style('white')
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    circles = plot_circles(data, ax=ax, cmap=cmap)
    cb = fig.colorbar(circles)
    cb.set_label('$r^2$', fontsize=16)
    ax.margins(0.1)
    plt.xlabel('Country trained on', fontsize=16)
    plt.ylabel('Country evaluated on', fontsize=16)
    plt.show()


def load_data(country_paths, survey, dimension):
    """
    Loads data for all surveys.
    """
    data = []
    for country_path in country_paths:
        X, y = load_and_reduce_country(country_path, survey, dimension)
        data.append((X, y))
    return data


def load_country_lsms(country_path):
    """
    Loads data for one LSMS country.
    """
    X = np.load(country_path + 'cluster_conv_features.npy')
    X_nl = np.load(country_path + 'cluster_nightlights.npy').reshape(-1, 1)
    y = np.load(country_path + 'cluster_consumptions.npy')
    hhs = np.load(country_path + 'cluster_households.npy')
    images = np.load(country_path + 'cluster_image_counts.npy')
    # Filter out single households and <10 images
    mask = np.logical_and((hhs >= 2), (images >= 10))
    X = X[mask]
    X_nl = X_nl[mask]
    y = y[mask]
    # Filter out 0 consumption clusters
    X = X[y > 0]
    X_nl = X_nl[y > 0]
    y = y[y > 0]
    y = np.log(y)
    return X, X_nl, y


def load_country_dhs(country_path):
    """
    Loads data for one DHS country.
    """
    X = np.load(country_path + 'cluster_conv_features.npy')
    X_nl = np.load(country_path + 'cluster_nightlights.npy').reshape(-1, 1)
    y = np.load(country_path + 'cluster_assets.npy')
    hhs = np.load(country_path + 'cluster_households.npy')
    images = np.load(country_path + 'cluster_image_counts.npy')
    # Filter out single households and <10 images
    mask = np.logical_and((hhs >= 2), (images >= 10))
    X = X[mask]
    X_nl = X_nl[mask]
    y = y[mask]
    return X, X_nl, y


def reduce_dimension(X, dimension):
    """
    Uses PCA to reduce dimensionality of features.
    """
    if dimension is not None:
        pca = PCA(n_components=dimension)
        X = pca.fit_transform(X)
    return X


def load_and_reduce_country(country_path, survey, dimension):
    """
    Loads data for one country and reduces the dimensionality of features.
    """
    if survey == 'lsms':
        X, _, y = load_country_lsms(country_path)
    elif survey == 'dhs':
        X, _, y = load_country_dhs(country_path)
    X = reduce_dimension(X, dimension)
    return X, y


def run_cv(X, y, k, k_inner, points, alpha_low, alpha_high, randomize=False):
    """
    Runs nested cross-validation to make predictions and compute r-squared.
    """
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s = np.zeros((k,))
    y_hat = np.zeros_like(y)
    kf = KFold(n_splits=k, shuffle=True)
    fold = 0
    for train_idx, test_idx in kf.split(X):
        r2s, y_hat, fold = evaluate_fold(
            X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
            randomize)
    return y_hat, r2s.mean()


def scale_features(X_train, X_test):
    """
    Scales features using StandardScaler.
    """
    X_scaler = StandardScaler(with_mean=True, with_std=False)
    X_train = X_scaler.fit_transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test


def train_and_predict_ridge(alpha, X_train, y_train, X_test):
    """
    Trains ridge model and predicts test set.
    """
    ridge = linear_model.Ridge(alpha)
    ridge.fit(X_train, y_train)
    y_hat = ridge.predict(X_test)
    return y_hat


def predict_inner_test_fold(X, y, y_hat, train_idx, test_idx, alpha):
    """
    Predicts inner test fold.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train, X_test = scale_features(X_train, X_test)
    y_hat[test_idx] = train_and_predict_ridge(alpha, X_train, y_train, X_test)
    return y_hat


def find_best_alpha(X, y, k_inner, alphas):
    """
    Finds the best alpha in an inner CV loop.
    """
    kf = KFold(n_splits=k_inner, shuffle=True)
    best_alpha = 0
    best_r2 = 0
    for idx, alpha in enumerate(alphas):
        y_hat = np.zeros_like(y)
        for train_idx, test_idx in kf.split(X):
            y_hat = predict_inner_test_fold(
                X, y, y_hat, train_idx, test_idx, alpha)
        r2 = stats.pearsonr(y, y_hat)[0] ** 2
        if r2 > best_r2:
            best_alpha = alpha
            best_r2 = r2
    print('best alpha', best_alpha)
    return best_alpha


def evaluate_fold(
    X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
        randomize):
    """
    Evaluates one fold of outer CV.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if randomize:
        random.shuffle(y_train)
    best_alpha = find_best_alpha(X_train, y_train, k_inner, alphas)
    X_train, X_test = scale_features(X_train, X_test)
    y_test_hat = train_and_predict_ridge(best_alpha, X_train, y_train, X_test)
    r2 = stats.pearsonr(y_test, y_test_hat)[0] ** 2
    r2s[fold] = r2
    y_hat[test_idx] = y_test_hat
    return r2s, y_hat, fold + 1


In [62]:
ss = StandardScaler()
x_hat = ss.fit_transform(x)

In [63]:
_, _, _, r2 = predict_consumption(x_hat, y_log)
r2

best alpha 599.4842503189409
best alpha 1668.100537200059
best alpha 599.4842503189409
best alpha 599.4842503189409
best alpha 1668.100537200059


0.4186669257724608

In [73]:
_, _, _, r2 = predict_consumption(x_hat, y)
r2

best alpha 12915.496650148827
best alpha 35938.13663804626
best alpha 1668.100537200059
best alpha 35938.13663804626
best alpha 12915.496650148827


0.19830535894547854

Quick and dirty hand-testing method

In [66]:
n_train = int(0.7*(len(x_hat))) # let's use 70% of the data for training
inds = np.arange(len(x_hat))
train_ind = np.random.choice(inds, n_train, replace=False)
valid_ind = np.delete(inds, train_ind)

train_x = x_hat[train_ind]
valid_x = x_hat[valid_ind]

train_y = y_log[train_ind]
valid_y = y_log[valid_ind]

In [67]:
train_y.mean(), valid_y.mean() # let's make sure we don't have vastly different distributions of train/test

(1.1443667920211187, 1.146268020299851)

In [68]:
ridge = linear_model.Ridge()
ridge.fit(train_x, train_y)
ridge.score(train_x, train_y)

0.889361380899413

In [69]:
# ??? what happened ???
# well, ridge regression is heavily affected by alpha (regularization) value
# Jean et. al. code finds a good alpha, which we do not do here
# if we use theirs, our ridge score will go up
ridge.score(valid_x, valid_y)

-0.7633108776629753

In [70]:
ridge.alpha

1.0

In [71]:
ridge2 = linear_model.Ridge(alpha=1000) # the best alphas printed suggest using a high alphsa
ridge2.fit(train_x, train_y)
ridge2.score(train_x, train_y)

0.5335034884094135

In [72]:
# this is closer to the r^2 using the paper's functions, as seen above
# they did other things like CV and identifying best alphas
# this was just meant to show a quick and dirty way of testing
ridge2.score(valid_x, valid_y)

0.35512950020455225